In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

1. add_datepart (not relevant)
2. train_cats
3. proc_df

In [3]:
df_trn_raw = pd.read_csv("data/titanic-train.csv", low_memory=False)

In [4]:
df_trn_raw.head(5).T

,0,1,2,3,4
PassengerId,1,2,3,4,5
Survived,0,1,1,1,0
Pclass,3,1,3,1,3
Name,"Braund, Mr. Owen Harris","Cumings, Mrs. John Bradley (Florence Briggs Th...","Heikkinen, Miss. Laina","Futrelle, Mrs. Jacques Heath (Lily May Peel)","Allen, Mr. William Henry"
Sex,male,female,female,female,male
Age,22,38,26,35,35
SibSp,1,1,0,1,0
Parch,0,0,0,0,0
Ticket,A/5 21171,PC 17599,STON/O2. 3101282,113803,373450
Fare,7.25,71.2833,7.925,53.1,8.05


In [5]:
del df_trn_raw["PassengerId"]
del df_trn_raw["Name"]
del df_trn_raw["Ticket"]

In [6]:
df_trn_raw["Cabin"] = df_trn_raw["Cabin"].fillna("N").str[0]

In [7]:
train_cats(df_trn_raw)

In [8]:
df_trn_raw.head(5).T

,0,1,2,3,4
Survived,0,1,1,1,0
Pclass,3,1,3,1,3
Sex,male,female,female,female,male
Age,22,38,26,35,35
SibSp,1,1,0,1,0
Parch,0,0,0,0,0
Fare,7.25,71.2833,7.925,53.1,8.05
Cabin,N,C,N,C,N
Embarked,S,C,S,S,S


In [9]:
df_trn_raw.dtypes

Survived       int64
Pclass         int64
Sex         category
Age          float64
SibSp          int64
Parch          int64
Fare         float64
Cabin       category
Embarked    category
dtype: object

In [10]:
X_trn, y_trn, nas = proc_df(df_trn_raw, "Survived")

In [11]:
def split_val(a, n): return a[:n], a[n:]

In [12]:
n_val = math.ceil(len(X_trn) * 0.25)
n_trn = 1 - n_val 

In [13]:
X_trn, X_val = split_val(X_trn, n_trn)
y_trn, y_val = split_val(y_trn, n_trn)

In [14]:
m = RandomForestClassifier(n_jobs=-1)
m.fit(X_trn, y_trn)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [15]:
def rmse(x, y): return math.sqrt(((x - y) ** 2).mean())

In [16]:
def rf_eval(m, X_trn, y_trn, X_val, y_val):
    print("RMSE trn: {}\nRMSE val: {}\nScore trn: {}\nScore val: {}".format(
        rmse(m.predict(X_trn), y_trn),
        rmse(m.predict(X_val), y_val),
        m.score(X_trn, y_trn),
        m.score(X_val, y_val)
    ))

In [17]:
rf_eval(m, X_trn, y_trn, X_val, y_val)

RMSE trn: 0.1729027652128102
RMSE val: 0.4026936331284146
Score trn: 0.9701046337817638
Score val: 0.8378378378378378


In [18]:
%ls data

titanic-sample-subm.csv  titanic-test.csv  titanic-train.csv


In [19]:
df_test_raw = pd.read_csv("data/titanic-test.csv", low_memory=False)

In [20]:
df_test = df_test_raw.copy()

del df_test["PassengerId"]
del df_test["Name"]
del df_test["Ticket"]

In [21]:
df_test["Cabin"] = df_test["Cabin"].fillna("N").str[0]

In [22]:
train_cats(df_test)

In [23]:
df_test.dtypes

Pclass         int64
Sex         category
Age          float64
SibSp          int64
Parch          int64
Fare         float64
Cabin       category
Embarked    category
dtype: object

In [24]:
df_test["_"] = "_"

In [25]:
X_test, _, nas = proc_df(df_test, "_")

In [26]:
X_trn.head(3)

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Age_na
0,3,2,22.0,1,0,7.2500,8,3,False
1,1,1,38.0,1,0,71.2833,3,1,False
2,3,1,26.0,0,0,7.9250,8,3,False


In [27]:
X_test.head(3)

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Age_na,Fare_na
0,3,2,34.5,0,0,7.8292,8,2,False,False
1,3,1,47.0,1,0,7.0000,8,3,False,False
2,2,2,62.0,0,0,9.6875,8,2,False,False


In [28]:
del X_test["Fare"]

In [29]:
preds_test = m.predict(X_test)

In [30]:
smp = pd.read_csv("data/titanic-sample-subm.csv")
smp.head(3)

,PassengerId,Survived
0,892,0
1,893,1
2,894,0


In [41]:
subm = pd.DataFrame({
    "PassengerId": df_test_raw["PassengerId"],
    "Survived": preds_test
})

In [42]:
subm.to_csv("data/titanic-rf-submission.csv", index=False)

In [43]:
from IPython.display import FileLinks

In [44]:
FileLinks("data/")

data/
  titanic-sample-subm.csv
  titanic-test.csv
  titanic-train.csv
  titanic-rf-submission.csv